# GNN Scalability Analysis: Static Graph Node Classification

This notebook demonstrates **scalable Graph Neural Network training** for Bitcoin fraud detection using **optimized neighborhood sampling strategies**. 

### 🔬 **Bitcoin Network Analysis**
Based on degree distribution where:
- 89.47% of nodes have ≤ 10 neighbors
- 95.29% of nodes have ≤ 25 neighbors
- Median degree: 2, Mean degree: 7
- Hub nodes: Few nodes with 30K+ neighbors

In [1]:
import sys
from pathlib import Path

ROOT = Path.cwd().parent.parent
sys.path.insert(0, str(ROOT))

from code_lib.temporal_node_classification_builder import (
    TemporalNodeClassificationBuilder,
    load_elliptic_data,
    prepare_observation_window_graphs
)

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv
from torch_geometric.loader import NeighborSampler
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from tqdm.notebook import tqdm
import time
torch.manual_seed(42)
np.random.seed(42)

## Configuration

In [2]:
from test_config import EXPERIMENT_CONFIG

CONFIG = EXPERIMENT_CONFIG.copy()

CONFIG['dropout'] = 0.3
CONFIG['learning_rate'] = 0.001
CONFIG['weight_decay'] = 1e-5
CONFIG['epochs'] = 400
CONFIG['patience'] = 20
CONFIG['observation_windows']: [3, 5, 7]

CONFIG['enable_sampling'] = True           # Enable neighborhood sampling
CONFIG['batch_size'] = 2048                # Mini-batch size for target nodes
CONFIG['num_workers'] = 4                  # Parallel data loading
CONFIG['normalize'] = True                 # L2 normalization

## Load Data & Create Splits

In [3]:
def remove_correlated_features(nodes_df, threshold=0.95, verbose=True):
    """
    Remove highly correlated features from nodes DataFrame.
    
    Args:
        nodes_df: DataFrame with node features
        threshold: Correlation threshold (default 0.95)
        verbose: Print removed features
    
    Returns:
        list of kept feature columns
    """
    # Identify feature columns (exclude address, Time step, class)
    exclude_cols = {'address', 'Time step', 'class'}
    feature_cols = [col for col in nodes_df.columns 
                    if col not in exclude_cols and 
                    pd.api.types.is_numeric_dtype(nodes_df[col])]
    
    # Compute correlation matrix on a sample (for speed)
    sample_size = min(10000, len(nodes_df))
    sample_df = nodes_df[feature_cols].sample(n=sample_size, random_state=42)
    corr_matrix = sample_df.corr().abs()
    
    # Find features to remove
    to_remove = set()
    
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] > threshold:
                # Remove the second feature (arbitrary choice)
                feature_to_remove = corr_matrix.columns[j]
                to_remove.add(feature_to_remove)
                if verbose:
                    print(f"Removing {feature_to_remove} (corr={corr_matrix.iloc[i, j]:.3f} with {corr_matrix.columns[i]})")
    
    # Keep features
    features_to_keep = [col for col in feature_cols if col not in to_remove]
    
    if verbose:
        print(f"\nFeature reduction summary:")
        print(f"  Original features: {len(feature_cols)}")
        print(f"  Removed features:  {len(to_remove)}")
        print(f"  Kept features:     {len(features_to_keep)}")
        print(f"  Reduction ratio:   {len(to_remove)/len(feature_cols)*100:.1f}%")
    
    return features_to_keep

print("✅ Feature correlation removal function defined!")

✅ Feature correlation removal function defined!


In [4]:
# Load data
print("📁 Loading Elliptic Bitcoin dataset...")
nodes_df, edges_df = load_elliptic_data(CONFIG['data_dir'], use_temporal_features=True)

print(f"📊 Dataset loaded:")
print(f"  Nodes: {nodes_df.shape[0]:,} rows × {nodes_df.shape[1]} columns")
print(f"  Edges: {edges_df.shape[0]:,} rows × {edges_df.shape[1]} columns")

# Remove highly correlated features to reduce dimensionality and improve performance
print(f"\n🔧 Removing highly correlated features (threshold=0.95)...")
kept_features = remove_correlated_features(nodes_df, threshold=0.95, verbose=True)

# Create temporal graph builder with reduced feature set
print(f"\n🏗️  Creating temporal graph builder with {len(kept_features)} features...")
builder = TemporalNodeClassificationBuilder(
    nodes_df=nodes_df,
    edges_df=edges_df,
    feature_cols=kept_features,  # Use only non-correlated features
    include_class_as_feature=False,
    add_temporal_features=True,
    use_temporal_edge_decay=False,
    cache_dir='../../graph_cache_reduced_features_fixed',  # New cache dir for reduced features
    use_cache=True,
    verbose=True
)

# Create temporal split
print(f"\n📊 Creating temporal train/val/test split...")
split = builder.get_train_val_test_split(
    train_timesteps=CONFIG['train_timesteps'],
    val_timesteps=CONFIG['val_timesteps'],
    test_timesteps=CONFIG['test_timesteps'],
    filter_unknown=True
)

print(f"\n✅ Data preparation complete:")
print(f"  Train: {len(split['train'])} nodes")
print(f"  Val:   {len(split['val'])} nodes")
print(f"  Test:  {len(split['test'])} nodes")
print(f"  Features used: {len(kept_features)} (after correlation removal)")

📁 Loading Elliptic Bitcoin dataset...
Loading trmporal features...
Loading node classes...
Loading edges...
📊 Dataset loaded:
  Nodes: 920,691 rows × 119 columns
  Edges: 2,868,964 rows × 187 columns

🔧 Removing highly correlated features (threshold=0.95)...
Removing out_num (corr=0.979 with in_num)
Removing in_fees_sum (corr=1.000 with in_total_fees)
Removing in_median_fees (corr=0.999 with in_mean_fees)
Removing in_fees_mean (corr=1.000 with in_mean_fees)
Removing in_fees_median (corr=0.999 with in_mean_fees)
Removing in_fees_mean (corr=0.999 with in_median_fees)
Removing in_fees_median (corr=1.000 with in_median_fees)
Removing in_total_BTC_sum (corr=1.000 with in_total_btc_in)
Removing in_in_BTC_max_sum (corr=0.978 with in_total_btc_in)
Removing in_in_BTC_total_sum (corr=1.000 with in_total_btc_in)
Removing in_out_BTC_max_sum (corr=0.982 with in_total_btc_in)
Removing in_out_BTC_total_sum (corr=1.000 with in_total_btc_in)
Removing in_median_btc_in (corr=0.997 with in_mean_btc_in)
Re

## Prepare Per-Node Graphs

Each node evaluated at t_first(v) + K.

In [5]:
device = torch.device(CONFIG['device'])

graphs = prepare_observation_window_graphs(
    builder,
    split['train'],
    split['val'],
    split['test'],
    K_values=CONFIG['observation_windows'],
    device=device
)


PREPARING OBSERVATION WINDOW GRAPHS (PER-NODE EVALUATION)

K = 1 (Each node evaluated at t_first + 1)

TRAIN split:
  Nodes to evaluate: 104,704
  Evaluation times: t=6 to t=27
  Unique graphs needed: 22
  ✅ Loaded cached graph from ../../graph_cache_reduced_features_fixed/graph_t6_metaTrue_classFalse_tempTrue_weightsFalse.pt
  ✅ Loaded cached graph from ../../graph_cache_reduced_features_fixed/graph_t7_metaTrue_classFalse_tempTrue_weightsFalse.pt
  ✅ Loaded cached graph from ../../graph_cache_reduced_features_fixed/graph_t8_metaTrue_classFalse_tempTrue_weightsFalse.pt
  ✅ Loaded cached graph from ../../graph_cache_reduced_features_fixed/graph_t9_metaTrue_classFalse_tempTrue_weightsFalse.pt
  ✅ Loaded cached graph from ../../graph_cache_reduced_features_fixed/graph_t10_metaTrue_classFalse_tempTrue_weightsFalse.pt
  ✅ Loaded cached graph from ../../graph_cache_reduced_features_fixed/graph_t11_metaTrue_classFalse_tempTrue_weightsFalse.pt
  ✅ Loaded cached graph from ../../graph_cache_re

In [9]:
class StandardGCN(nn.Module):
    """
    Standard GCN without sampling - traditional full graph approach.
    """
    def __init__(self, num_features, hidden_dim, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)
        self.dropout = dropout
        print(f"Standard GCN initialized (no sampling)")
        
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x


class SampledGCN(nn.Module):
    """
    GCN with neighborhood sampling for scalability.
    """
    def __init__(self, num_features, hidden_dim, num_classes, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)
        self.dropout = dropout
        print(f"Sampled GCN initialized (with neighborhood sampling)")
        
    def forward(self, x, edge_index):
        # Standard forward for full graphs
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x
    
    def forward_sampled(self, x, adjs):
        """Forward pass for sampled subgraphs from NeighborSampler."""
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]
            if i == 0:
                x = self.conv1(x, edge_index)
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
            else:
                x = self.conv2(x, edge_index)
            x = x[:size[1]]  # Keep only target nodes
        return x


# Model factory function
def create_model(model_type, num_features, hidden_dim, num_classes, 
                dropout=0.5, normalize=True):
    """Factory function to create different model types."""
    if model_type == "standard_gcn":
        return StandardGCN(num_features, hidden_dim, num_classes, dropout)
    elif model_type == "sampled_gcn":
        return SampledGCN(num_features, hidden_dim, num_classes, dropout)
    else:
        raise ValueError(f"Unknown model type: {model_type}")

print("✅ All model classes defined!")
print("Available models: standard_gcn, sampled_gcn")

✅ All model classes defined!
Available models: standard_gcn, sampled_gcn


In [10]:
def calculate_adaptive_sampling_strategy(graph):
    """
    Calculate adaptive sampling strategy based on node degrees.
    - Hubs (top 1%): 30 neighbors first layer, 15 neighbors second layer
    - Medium (next 5%): 12 neighbors first layer, 6 neighbors second layer  
    - Low (remaining 94%): 2 neighbors first layer, 1 neighbor second layer
    
    Args:
        graph: PyTorch Geometric graph
    
    Returns:
        Dict with sampling strategies for different node types
    """
    from torch_geometric.utils import degree
    
    # Calculate node degrees
    degrees = degree(graph.edge_index[0], graph.num_nodes)
    
    # Calculate thresholds - top 1% are hubs, next 5% are medium
    hub_threshold = torch.quantile(degrees, 0.99).item()  # Top 1%
    medium_threshold = torch.quantile(degrees, 0.95).item()  # Top 6% (1% hubs + 5% medium)
    
    # Create fixed sampling strategies
    strategies = {
        'low_degree': [2, 2],
        'medium_degree': [8, 2],
        'high_degree': [20, 5]
    }
    
    # Count nodes in each category
    high_degree_count = (degrees >= hub_threshold).sum().item()
    medium_degree_count = ((degrees >= medium_threshold) & (degrees < hub_threshold)).sum().item()
    low_degree_count = (degrees < medium_threshold).sum().item()
    
    analysis = {
        'total_nodes': graph.num_nodes,
        'hub_threshold': hub_threshold,
        'medium_threshold': medium_threshold,
        'max_degree': degrees.max().item(),
        'min_degree': degrees.min().item(),
        'low_degree_nodes': low_degree_count,
        'medium_degree_nodes': medium_degree_count, 
        'high_degree_nodes': high_degree_count,
        'strategies': strategies
    }
    
    return analysis


def create_hub_aware_samplers(graphs_dict, config, model_type):
    """
    Create NeighborSamplers with hub-aware adaptive sampling.
    Uses specific sampling strategies: Hubs (top 1%) get [30,15], Medium (next 5%) get [12,6], Low get [2,1].
    """
    use_sampling = model_type in ["sampled_gcn"] and config['enable_sampling']
    
    if not use_sampling:
        return {'graphs': graphs_dict, 'samplers': None, 'target_nodes': None, 'adaptive_info': None}
    else:
        samplers = {}
        target_nodes_dict = {}
        adaptive_analyses = {}
                
        for eval_t, graph in graphs_dict.items():
            # Analyze graph and determine adaptive strategies
            adaptive_analysis = calculate_adaptive_sampling_strategy(graph)
            adaptive_analyses[eval_t] = adaptive_analysis
            
            # Use medium-degree strategy as the default sampler (balanced approach)
            sampling_strategy = adaptive_analysis['strategies']['medium_degree']
            
            # Create target nodes (staying on CPU for NeighborSampler)
            target_nodes = torch.where(graph.eval_mask)[0].cpu()
            target_nodes_dict[eval_t] = target_nodes
            
            # Create sampler with adaptive strategy
            from torch_geometric.loader import NeighborSampler
            sampler = NeighborSampler(
                graph.edge_index.cpu(),
                sizes=sampling_strategy,  # Use adaptive sampling sizes
                batch_size=config['batch_size'],
                shuffle=True,
                num_workers=config.get('num_workers', 4)
            )
            
            samplers[eval_t] = sampler
        
        return {
            'graphs': graphs_dict,
            'samplers': samplers,
            'target_nodes': target_nodes_dict,
            'adaptive_info': adaptive_analyses
        }


def train_epoch_with_hub_aware_samplers(model, sampler_data, optimizer, criterion, config, model_type):
    """
    Enhanced training function with hub-aware sampling insights.
    """
    model.train()
    total_loss = 0
    total_correct = 0 
    total_samples = 0
    
    total_sampling_time = 0
    total_forward_backward_time = 0
    
    use_sampling = model_type in ["sampled_gcn"] and config['enable_sampling']
    
    if not use_sampling:
        # Standard full graph training
        for eval_t, graph in sampler_data['graphs'].items():
            fb_start = time.time()
            logits = model(graph.x, graph.edge_index)
            loss = criterion(logits[graph.eval_mask], graph.y[graph.eval_mask])
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_forward_backward_time += time.time() - fb_start
            
            pred = logits[graph.eval_mask].argmax(dim=1)
            correct = (pred == graph.y[graph.eval_mask]).sum().item()
            
            total_loss += loss.item()
            total_correct += correct
            total_samples += graph.eval_mask.sum().item()
    else:
        # Hub-aware sampled training using pre-built samplers
        graphs = sampler_data['graphs']
        samplers = sampler_data['samplers']
        target_nodes_dict = sampler_data['target_nodes']
        
        for eval_t in graphs.keys():
            graph = graphs[eval_t]
            sampler = samplers[eval_t]
            target_nodes = target_nodes_dict[eval_t]
            
            # Sample subgraphs (with hub-aware sampling sizes)
            sampling_start = time.time()
            for batch_size, n_id, adjs in [sampler.sample(target_nodes)]:
                total_sampling_time += time.time() - sampling_start
                
                # Extract features for sampled nodes
                x_batch = graph.x[n_id].to(graph.x.device)
                y_batch = graph.y[target_nodes].to(graph.y.device)
                
                # Convert adjacency info for model
                adjs = [(adj.edge_index.to(graph.x.device), adj.e_id, adj.size) for adj in adjs]
                
                # Forward and backward pass
                fb_start = time.time()
                if hasattr(model, 'forward_sampled'):
                    logits = model.forward_sampled(x_batch, adjs)
                else:
                    # Use first adjacency for simple models
                    edge_index = adjs[0][0] if adjs else torch.empty((2, 0), device=graph.x.device)
                    logits = model(x_batch, edge_index)
                
                # Loss only on target nodes (first batch_size nodes)
                target_logits = logits[:batch_size]
                loss = criterion(target_logits, y_batch)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_forward_backward_time += time.time() - fb_start
                
                pred = target_logits.argmax(dim=1)
                correct = (pred == y_batch).sum().item()
                
                total_loss += loss.item()
                total_correct += correct
                total_samples += batch_size
    
    # Store timing info
    train_epoch_with_hub_aware_samplers.last_sampling_time = total_sampling_time
    train_epoch_with_hub_aware_samplers.last_forward_backward_time = total_forward_backward_time
    
    if use_sampling:
        avg_loss = total_loss / max(total_samples // config['batch_size'], 1) if total_samples > 0 else 0
    else:
        avg_loss = total_loss / len(sampler_data['graphs']) if len(sampler_data['graphs']) > 0 else 0
        
    avg_acc = total_correct / total_samples if total_samples > 0 else 0
    
    return avg_loss, avg_acc


def evaluate_with_hub_aware_samplers(model, sampler_data, config, model_type):
    """
    Enhanced evaluation with hub-aware sampling insights.
    """
    model.eval()
    all_preds = []
    all_labels = []
    all_probs = []
    
    use_sampling = model_type in ["sampled_gcn"] and config['enable_sampling']
    
    with torch.no_grad():
        if not use_sampling:
            # Standard evaluation
            for eval_t, graph in sampler_data['graphs'].items():
                logits = model(graph.x, graph.edge_index)
                pred = logits[graph.eval_mask].argmax(dim=1).cpu().numpy()
                true = graph.y[graph.eval_mask].cpu().numpy()
                probs = F.softmax(logits[graph.eval_mask], dim=1)[:, 1].cpu().numpy()
                
                all_preds.append(pred)
                all_labels.append(true)
                all_probs.append(probs)
        else:
            # Hub-aware sampled evaluation
            graphs = sampler_data['graphs']
            samplers = sampler_data['samplers']
            target_nodes_dict = sampler_data['target_nodes']
            
            for eval_t in graphs.keys():
                graph = graphs[eval_t]
                sampler = samplers[eval_t]
                target_nodes = target_nodes_dict[eval_t]
                
                for batch_size, n_id, adjs in [sampler.sample(target_nodes)]:
                    x_batch = graph.x[n_id].to(graph.x.device)
                    adjs = [(adj.edge_index.to(graph.x.device), adj.e_id, adj.size) for adj in adjs]
                    
                    if hasattr(model, 'forward_sampled'):
                        logits = model.forward_sampled(x_batch, adjs)
                    else:
                        edge_index = adjs[0][0] if adjs else torch.empty((2, 0), device=graph.x.device)
                        logits = model(x_batch, edge_index)
                    
                    target_logits = logits[:batch_size]
                    pred = target_logits.argmax(dim=1).cpu().numpy()
                    probs = F.softmax(target_logits, dim=1)[:, 1].cpu().numpy()
                    
                    all_preds.append(pred)
                    all_labels.append(graph.y[target_nodes].cpu().numpy())
                    all_probs.append(probs)
    
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    all_probs = np.concatenate(all_probs)
    
    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_labels, all_preds, average='binary', pos_label=1, zero_division=0
    )
    auc = roc_auc_score(all_labels, all_probs) if len(np.unique(all_labels)) > 1 else 0.5
    
    return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1': f1, 'auc': auc}

In [11]:
model_types = [
    "sampled_gcn",
    "standard_gcn",
]

model_names = {
    "standard_gcn": "Standard GCN",
    "sampled_gcn": f"GCN + Hub-Aware Sampling"
}

# Store results for each model type and K value
all_results = {}
all_models = {}
all_timings = {}

for model_type in model_types:
    print(f"\n{'='*80}")
    if model_type.startswith('sampled'):
        print(f"TRAINING SAMPLING MODEL: {model_names[model_type]}")
    else:
        print(f"TRAINING STANDARD MODEL: {model_names[model_type]}")
    print('='*80)
    
    all_results[model_type] = {}
    all_models[model_type] = {}
    all_timings[model_type] = {}
    
    for K in CONFIG['observation_windows']:
        print(f"\nModel: {model_names[model_type]} | K={K}")
        
        # Start total timing for this configuration
        total_start_time = time.time()
        
        train_graphs = graphs[K]['train']['graphs']
        val_graphs = graphs[K]['val']['graphs']
        test_graphs = graphs[K]['test']['graphs']
        
        # Time model initialization
        init_start_time = time.time()
        num_features = list(train_graphs.values())[0].x.shape[1]
        model = create_model(
            model_type=model_type,
            num_features=num_features,
            hidden_dim=CONFIG['hidden_dim'],
            num_classes=2,
            dropout=CONFIG['dropout'],
            normalize=CONFIG['normalize']
        ).to(device)
        
        optimizer = torch.optim.Adam(
            model.parameters(),
            lr=CONFIG['learning_rate'],
            weight_decay=CONFIG['weight_decay']
        )
        init_time = time.time() - init_start_time
        
        # Compute class weights
        all_train_labels = []
        for g in train_graphs.values():
            all_train_labels.append(g.y[g.eval_mask].cpu())
        all_train_labels = torch.cat(all_train_labels).long()
        
        class_counts = torch.bincount(all_train_labels)
        class_weights = torch.sqrt(1.0 / class_counts.float())
        class_weights = class_weights / class_weights.sum() * 2.0
        class_weights = class_weights.to(device)
        
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        
        # Training loop with comprehensive timing tracking
        best_val_f1 = 0
        patience_counter = 0
        best_model_state = None
        
        # Universal timing tracking for all models
        epoch_times = []
        training_times = []  # Time spent on training per epoch
        validation_times = []  # Time spent on validation per epoch
        train_losses = []  # Track training losses
        
        # Start training timing
        training_start_time = time.time()
        
        # Check if this is a sampling model
        is_sampling_model = model_type.startswith('sampled') and CONFIG['enable_sampling']
        
        # CREATE HUB-AWARE SAMPLERS ONCE FOR ENTIRE TRAINING (ULTRA-OPTIMIZATION!)
        print(f"   🔧 Creating hub-aware adaptive samplers once for entire training...")
        sampler_creation_start = time.time()
        train_sampler_data = create_hub_aware_samplers(train_graphs, CONFIG, model_type)
        val_sampler_data = create_hub_aware_samplers(val_graphs, CONFIG, model_type)
        test_sampler_data = create_hub_aware_samplers(test_graphs, CONFIG, model_type)
        sampler_creation_time = time.time() - sampler_creation_start
        

        print(f"   {'Epoch':<5} | {'Loss':<8} | {'Train F1':<8} | {'Val F1':<8} | {'Epoch Time':<10} | {'Details'}")
        print(f"   {'─' * 75}")
        
        pbar = tqdm(range(CONFIG['epochs']), desc=f"{model_names[model_type]} K={K}")
        
        for epoch in pbar:
            # Time individual epoch
            epoch_start_time = time.time()
            
            # TRAINING PHASE TIMING
            train_start = time.time()
            
            if is_sampling_model:
                # Ultra-optimized hub-aware training using adaptive sampling
                train_loss, train_acc = train_epoch_with_hub_aware_samplers(
                    model, train_sampler_data, optimizer, criterion, CONFIG, model_type
                )
            else:
                # Standard training for non-sampling strategiesmodels
                train_loss, train_acc = train_epoch_with_hub_aware_samplers(
                    model, train_sampler_data, optimizer, criterion, CONFIG, model_type
                )
            
            training_time_this_epoch = time.time() - train_start
            training_times.append(training_time_this_epoch)
            train_losses.append(train_loss)  # Track loss
            
            # VALIDATION PHASE TIMING (every 5 epochs)
            validation_time_this_epoch = 0
            if (epoch + 1) % 5 == 0:
                val_start = time.time()
                val_metrics = evaluate_with_hub_aware_samplers(model, val_sampler_data, CONFIG, model_type)
                train_metrics = evaluate_with_hub_aware_samplers(model, train_sampler_data, CONFIG, model_type)
                validation_time_this_epoch = time.time() - val_start
                validation_times.append(validation_time_this_epoch)
                
                epoch_time = time.time() - epoch_start_time
                epoch_times.append(epoch_time)
                
                # Print progress with universal timing breakdown
                details = f"Train:{training_time_this_epoch:.2f}s"
                
                print(f"   {epoch+1:<5} | {train_loss:<8.4f} | {train_metrics['f1']:<8.4f} | {val_metrics['f1']:<8.4f} | {details}")
                
                pbar.set_postfix({
                    'loss': f"{train_loss:.4f}",
                    'train_f1': f"{train_metrics['f1']:.4f}",
                    'val_f1': f"{val_metrics['f1']:.4f}",
                    'epoch_time': f"{epoch_time:.2f}s"
                })
                
                if val_metrics['f1'] > best_val_f1:
                    best_val_f1 = val_metrics['f1']
                    patience_counter = 0
                    best_model_state = model.state_dict().copy()
                else:
                    patience_counter += 1
                    
                if patience_counter >= CONFIG['patience']:
                    print(f"\n Early stopping at epoch {epoch+1} (patience={CONFIG['patience']})")
                    break
        
        training_time = time.time() - training_start_time
        
        # Load best model and evaluate on both validation and test sets
        if best_model_state is not None:
            model.load_state_dict(best_model_state)
        
        # Time final evaluation using hub-aware samplers
        final_eval_start = time.time()
        train_metrics = evaluate_with_hub_aware_samplers(model, train_sampler_data, CONFIG, model_type)
        val_metrics = evaluate_with_hub_aware_samplers(model, val_sampler_data, CONFIG, model_type)
        test_metrics = evaluate_with_hub_aware_samplers(model, test_sampler_data, CONFIG, model_type)
        final_eval_time = time.time() - final_eval_start
        
        total_time = time.time() - total_start_time
        
        # Store comprehensive timing information with universal train/validation split
        timing_info = {
            'total_time': total_time,
            'init_time': init_time,
            'sampler_creation_time': sampler_creation_time,
            'total_training_time': training_time,
            'final_eval_time': final_eval_time,
            'avg_epoch_time': np.mean(epoch_times) if epoch_times else 0,
            'total_epochs': len(epoch_times),
            'final_loss': train_losses[-1] if train_losses else 0,
            # Universal training/validation timing breakdown
            'avg_training_time_per_epoch': np.mean(training_times) if training_times else 0,
            'total_validation_phase_time': np.sum(validation_times) if validation_times else 0,
            'training_percentage': (np.sum(training_times) / training_time * 100) if training_times and training_time > 0 else 0,
            'validation_percentage': (np.sum(validation_times) / training_time * 100) if validation_times and training_time > 0 else 0
        }
        
        all_timings[model_type][K] = timing_info
        
        # Enhanced display with loss information and universal timing breakdown
        print(f"\n   FINAL RESULTS:")
        print(f"   Train: F1={train_metrics['f1']:.4f}, AUC={train_metrics['auc']:.4f}, Acc={train_metrics['accuracy']:.4f}, Loss={timing_info['final_loss']:.4f}")
        print(f"   Val:   F1={val_metrics['f1']:.4f}, AUC={val_metrics['auc']:.4f}, Acc={val_metrics['accuracy']:.4f}")
        print(f"   Test:  F1={test_metrics['f1']:.4f}, AUC={test_metrics['auc']:.4f}, Acc={test_metrics['accuracy']:.4f}")
        
        # Show universal timing breakdown with hub analysis
        if is_sampling_model:
            print(f"   🔧 Hub-Aware Samplers: {sampler_creation_time:.2f}s (intelligent adaptive sampling!)")
        
        # Universal training/validation timing breakdown (applies to all models)
        if training_times or validation_times:
            if validation_times:
                print(f"      • Validation Phase: {timing_info['total_validation_phase_time']:.1f}s ({timing_info['validation_percentage']:.1f}% of training)")
            print(f"      • Avg per epoch: Training={timing_info['avg_training_time_per_epoch']:.2f}s", end="")
        all_results[model_type][K] = {
            'train': train_metrics, 
            'val': val_metrics, 
            'test': test_metrics,
            'timing': timing_info
        }
        all_models[model_type][K] = model

print(f"\n{'='*80}")
print("ULTRA-OPTIMIZED MODEL TRAINING COMPLETE!")
print("Samplers created ONCE for maximum efficiency!")
print('='*80)


TRAINING SAMPLING MODEL: GCN + Hub-Aware Sampling

Model: GCN + Hub-Aware Sampling | K=1
Sampled GCN initialized (with neighborhood sampling)
   🔧 Creating hub-aware adaptive samplers once for entire training...
   Epoch | Loss     | Train F1 | Val F1   | Epoch Time | Details
   ───────────────────────────────────────────────────────────────────────────


GCN + Hub-Aware Sampling K=1:   0%|          | 0/400 [00:00<?, ?it/s]

   5     | 61577.5348 | 0.0000   | 0.0000   | Train:0.18s
   10    | 43856.1477 | 0.0839   | 0.0833   | Train:0.18s
   15    | 41729.7219 | 0.1702   | 0.1622   | Train:0.18s
   20    | 24138.0344 | 0.2513   | 0.2923   | Train:0.19s
   25    | 27639.1446 | 0.3329   | 0.2799   | Train:0.18s
   30    | 41396.1199 | 0.1896   | 0.1742   | Train:0.18s
   35    | 20369.2356 | 0.2557   | 0.2616   | Train:0.19s
   40    | 23104.1010 | 0.2626   | 0.2972   | Train:0.19s
   45    | 11590.1577 | 0.2627   | 0.3028   | Train:0.19s
   50    | 10906.6191 | 0.3009   | 0.4074   | Train:0.18s
   55    | 11732.8206 | 0.3229   | 0.3800   | Train:0.19s
   60    | 12084.5748 | 0.3120   | 0.4239   | Train:0.17s
   65    | 9375.4122 | 0.3310   | 0.4309   | Train:0.17s
   70    | 15595.0535 | 0.3939   | 0.3976   | Train:0.19s
   75    | 5072.1798 | 0.3468   | 0.3717   | Train:0.17s
   80    | 24628.1896 | 0.3861   | 0.3737   | Train:0.17s
   85    | 14702.3040 | 0.3662   | 0.4222   | Train:0.18s
   90    | 7536.

GCN + Hub-Aware Sampling K=3:   0%|          | 0/400 [00:00<?, ?it/s]

   5     | 37052.6434 | 0.1001   | 0.0975   | Train:0.17s
   10    | 32279.8306 | 0.1434   | 0.1632   | Train:0.18s
   15    | 33657.1293 | 0.2290   | 0.2559   | Train:0.17s
   20    | 24339.2261 | 0.2485   | 0.2789   | Train:0.17s
   25    | 15539.1328 | 0.2998   | 0.3705   | Train:0.19s
   30    | 21674.3798 | 0.2494   | 0.2775   | Train:0.19s
   35    | 16646.4706 | 0.2755   | 0.3348   | Train:0.17s
   40    | 12861.1737 | 0.3032   | 0.3970   | Train:0.19s
   45    | 23077.4868 | 0.3818   | 0.3913   | Train:0.19s
   50    | 7889.7400 | 0.2868   | 0.3950   | Train:0.20s
   55    | 24261.8546 | 0.2759   | 0.3789   | Train:0.19s
   60    | 5489.8863 | 0.3074   | 0.3400   | Train:0.18s
   65    | 5600.4217 | 0.3189   | 0.3855   | Train:0.19s
   70    | 18561.9342 | 0.3686   | 0.4006   | Train:0.18s
   75    | 4791.7257 | 0.3066   | 0.4167   | Train:0.17s
   80    | 3098.3951 | 0.3315   | 0.3569   | Train:0.17s
   85    | 7996.2305 | 0.3776   | 0.4222   | Train:0.18s
   90    | 2627.0045

...

   5     | 107359.0556 | 0.0000   | 0.0000   | Train:0.29s
   10    | 90126.0055 | 0.0000   | 0.0000   | Train:0.29s
   15    | 64300.7272 | 0.0048   | 0.0147   | Train:0.29s
   20    | 83410.5362 | 0.0985   | 0.1148   | Train:0.30s
   25    | 100869.3650 | 0.1304   | 0.1504   | Train:0.29s
   30    | 86438.0979 | 0.2104   | 0.3290   | Train:0.29s
   35    | 77263.6370 | 0.2442   | 0.3323   | Train:0.29s
   40    | 68483.2937 | 0.2816   | 0.4045   | Train:0.29s
   45    | 59830.5387 | 0.2879   | 0.4070   | Train:0.29s
   50    | 53735.9087 | 0.2960   | 0.4113   | Train:0.29s
   55    | 93559.2822 | 0.2961   | 0.4127   | Train:0.29s
   60    | 69203.8448 | 0.3052   | 0.3881   | Train:0.29s
   65    | 55194.9578 | 0.3187   | 0.3966   | Train:0.29s
   70    | 61528.4937 | 0.3189   | 0.3956   | Train:0.29s
   75    | 73019.7436 | 0.3250   | 0.3982   | Train:0.29s
   80    | 79089.5169 | 0.3576   | 0.4104   | Train:0.29s
   85    | 63658.5590 | 0.3406   | 0.3800   | Train:0.29s
   90    | 5

Standard GCN K=5:   0%|          | 0/400 [00:00<?, ?it/s]

   5     | 120380.4353 | 0.0000   | 0.0000   | Train:0.31s
   10    | 59624.2837 | 0.1750   | 0.1711   | Train:0.31s
   15    | 122598.3028 | 0.0000   | 0.0000   | Train:0.31s
   20    | 87312.5943 | 0.3220   | 0.4050   | Train:0.31s
   25    | 68092.9057 | 0.2909   | 0.3936   | Train:0.32s
   30    | 46955.7226 | 0.3049   | 0.3596   | Train:0.31s
   35    | 92402.2523 | 0.2655   | 0.3774   | Train:0.31s
   40    | 60115.4649 | 0.3058   | 0.3322   | Train:0.31s
   45    | 53277.6104 | 0.3177   | 0.3914   | Train:0.31s
   50    | 104368.8704 | 0.2534   | 0.3752   | Train:0.31s
   55    | 37040.7036 | 0.3010   | 0.3319   | Train:0.31s
   60    | 37291.1923 | 0.3163   | 0.3499   | Train:0.31s
   65    | 36446.9092 | 0.3294   | 0.3507   | Train:0.31s
   70    | 36654.9819 | 0.3138   | 0.3407   | Train:0.31s
   75    | 36334.7922 | 0.3474   | 0.3496   | Train:0.31s
   80    | 33544.5985 | 0.3414   | 0.3636   | Train:0.31s
   85    | 35670.1501 | 0.3580   | 0.3437   | Train:0.31s
   90    | 

Standard GCN K=7:   0%|          | 0/400 [00:00<?, ?it/s]

   5     | 53329.2773 | 0.0988   | 0.1080   | Train:0.33s
   10    | 41126.8267 | 0.2401   | 0.2651   | Train:0.34s
   15    | 41062.6192 | 0.2483   | 0.2775   | Train:0.33s
   20    | 32053.5088 | 0.3033   | 0.3839   | Train:0.33s
   25    | 25801.3603 | 0.3058   | 0.4095   | Train:0.33s
   30    | 16990.5534 | 0.3242   | 0.3780   | Train:0.33s
   35    | 13718.9844 | 0.3038   | 0.4153   | Train:0.33s
   40    | 31969.3183 | 0.1395   | 0.1432   | Train:0.33s
   45    | 12028.8028 | 0.3223   | 0.5819   | Train:0.33s
   50    | 5553.9334 | 0.3176   | 0.5128   | Train:0.34s
   55    | 6602.9390 | 0.3295   | 0.3470   | Train:0.33s
   60    | 6558.7972 | 0.3396   | 0.5841   | Train:0.33s
   65    | 12915.1411 | 0.2645   | 0.5664   | Train:0.33s
   70    | 1707.5233 | 0.3359   | 0.3419   | Train:0.33s
   75    | 13928.5814 | 0.3951   | 0.5797   | Train:0.34s
   80    | 5450.0073 | 0.3257   | 0.4433   | Train:0.34s
   85    | 1111.1648 | 0.3385   | 0.4623   | Train:0.33s
   90    | 930.8967 